Import the libraries needed to manipulate data

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import pyodbc
plt.style.use('ggplot')
%matplotlib inline

connect to the DailyMLResults database and get the data needed for this run

In [2]:
import pyodbc
conn = pyodbc.connect(r'Driver={SQL Server};Server=uatdotdatarpt;Database=DailyMLResults;Trusted_Connection=yes;OPTION=3;applicationintent=readonly;')
data = pd.read_sql("""SELECT Title, SourceLink, Modified_IsSecurity FROM MSSecurity_3 WITH (NOLOCK)
        WHERE Type IN ('bug', 'Compliance Issue', 'Defect', 'Issue', 'RDBug')
        AND AreaPath NOT LIKE '%RD\OneComplianceTest%' 
        AND AreaPath NOT LIKE '%\ClientManagement%'
        AND (IsDataDraft = 0 OR IsDataDraft IS NULL) 

UNION
SELECT Title, SourceLink, Modified_IsSecurity FROM Not_MSSecurity_3 WITH (NOLOCK)
        WHERE Type IN ('bug', 'Compliance Issue', 'Defect', 'Issue', 'RDBug')
        AND AreaPath NOT LIKE '%RD\OneComplianceTest%' 
        AND AreaPath NOT LIKE '%\ClientManagement%'
        AND (IsDataDraft = 0 OR IsDataDraft IS NULL) 

UNION
SELECT Title, SourceLink, Modified_IsSecurity FROM Not_Labeled_3 WITH (NOLOCK)
        WHERE Type IN ('bug', 'Compliance Issue', 'Defect', 'Issue', 'RDBug')
        AND AreaPath NOT LIKE '%RD\OneComplianceTest%' 
        AND AreaPath NOT LIKE '%\ClientManagement%'
        AND (IsDataDraft = 0 OR IsDataDraft IS NULL ) """, conn)
conn.close()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8863375 entries, 0 to 8863374
Data columns (total 3 columns):
Title                  object
SourceLink             object
Modified_IsSecurity    bool
dtypes: bool(1), object(2)
memory usage: 143.7+ MB


Define functions to clean the data

In [3]:
#Rmoves aldd urls from a title
import re
def clean_review_html(raw_data):
    text = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\$|\?|\=|\&|\%)*\b',' ', raw_data, flags=re.MULTILINE)
    letters_only=re.sub('\s+',' ', text)
    return letters_only

#removes all paths from the titles
def clean_review_path(raw_data):
    raw_text = re.sub(r'(\w+):(\\[A-Z0-9a-z_-]*\s*[A-Za-z0-9_-]*)?(\\[A-Za-z0-9_-]*\s*[A-Za-z0-9_-]*)*\b', '', raw_data, flags=re.MULTILINE)
    removed_path = re.sub(r'(\w+):(\[A-Z0-9a-z_-]*\s*[A-Za-z0-9_-]*)?(\[A-Za-z0-9_-]*\s*[A-Za-z0-9_-]*)*\b', '', raw_text, flags=re.MULTILINE)
    return removed_path

#Remove all punctuation marks from a string including underscores.
def remove_punctuation(text):
    return re.sub(r"[\\]", ' ',(' '.join(re.sub(r"[!\"#$%&'()*+,-./:;<=>?@[\]^_`{|}~]", ' ',text).split())))

#remove all words attached to '/' or '\'. this will remove all links that have not been removed
def remove_paths_urls(text):#'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
    return ' '.join([word for word in text.split() if '\\' not in word and '/' not in word])

In [4]:
# Split Title at upper case takes IAmTHEBoy you know => I Am THE Boy you know.
def split_on_uppercase(s, keep_contiguous=True):
    """  Args:
        s (str): string
        keep_contiguous (bool): flag to indicate we want to 
                                keep contiguous uppercase chars together
    """

    string_length = len(s)
    is_lower_around = (lambda: s[i-1].islower() or string_length > (i + 1) and s[i + 1].islower())
    start = 0
    parts = []
    for i in range(1, string_length):
        if s[i].isupper() and (not keep_contiguous or is_lower_around()):
            parts.append(s[start: i])
            start = i
    parts.append(s[start:])

    return " ".join(parts)
print(split_on_uppercase('TheLongANDANdeWindingRoad', True))  

The Long ANDA Nde Winding Road


In [13]:
#remove all integers from titles.
def remove_integers(s):
    return re.sub(' \d+ ', ' ', s)
print(remove_punctuation('MSAzureIntune-Svc-IWPortal_Release_1608_CTIP'))
remove_integers(remove_punctuation('MSAzureIntune-Svc-IWPortal_Release_1608_CTIP'))

MSAzureIntune Svc IWPortal Release 1608 CTIP


'MSAzureIntune Svc IWPortal Release CTIP'

Following our discussions with Alok, we had decided to make all instances of PoliCheck to 'policheck'. here, I create a function to do so.

In [5]:
def deal_with_policheck(title):
    return re.sub('(PoliCheck)|(POLICHECK)|(poliCheck)', ' policheck ', title)
deal_with_policheck('''[ DotNet-CoreFx-Trusted_master ][ PoliCheck ] - Defect :''')

'[ DotNet-CoreFx-Trusted_master ][  policheck  ] - Defect :'

In [6]:
data['Title'] = data['Title'].apply(str)

In [7]:

data['Title'] = data['Title'].apply(deal_with_policheck)

In [8]:
data['No Urls'] = data['Title'].apply(clean_review_html)

In [9]:
data['No Paths'] = data['No Urls'].apply(clean_review_path)

In [10]:
data['No Links and Paths'] = data['No Paths'].apply(remove_paths_urls)
#This function is just to ensure that if some paths and urls were not removedby the 
#regular expression, they must be removed now

In [11]:
data['Split Upper'] = data['No Links and Paths'].apply(split_on_uppercase)

In [14]:
data['No Punctuation'] = data['Split Upper'].apply(remove_punctuation)

In [15]:
data['Clean Title'] = data['No Punctuation'].apply(remove_integers)

# Model Building¶

We are going to build a bunch of models starting with unigrams and ending with trigrams (3=grams)

We fist import the neccessary libraries, build a tfidf and a model.

We end by validating the model on the validation set and getting output on the test set

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib

In [18]:
X_12, y_12 = data['Clean Title'], data['Modified_IsSecurity']
tv_12 = TfidfVectorizer(min_df = 0.00009, max_df = 0.5, use_idf = True, stop_words = 'english', ngram_range = (1,2))
X_12 = tv_12.fit_transform(X_12)
X_train_12, X_12_0, y_train_12, y_12_0 = train_test_split(X_12, y_12, test_size = 0.2, random_state = 42)
X_test_12, X_val_12, y_test_12, y_val_12 = train_test_split(X_12_0, y_12_0, test_size = 0.5, random_state = 42)
vocab_12 = tv_12.get_feature_names()
print(len(vocab_12))

15179


Train a logistic regression model and Get predictions on the training, validation and whole dataset.

In [19]:
model_lr_12 = LogisticRegression(n_jobs = -1)
model_lr_12.fit(X_train_12, y_train_12)

C:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [20]:
print('*'*15 + ' Results on Test set ' +'*'*15)
pred_12 = model_lr_12.predict(X_test_12)

score = metrics.accuracy_score(y_test_12, pred_12)
print("accuracy:   %0.3f" % score)

f1_score = metrics.f1_score(y_test_12, pred_12)
print("f1_score:   %0.3f" % f1_score)

precision_score = metrics.precision_score(y_test_12, pred_12)
print("precision_score:   %0.3f" % precision_score)

recall_score = metrics.recall_score(y_test_12, pred_12)
print("recall_score:   %0.3f" % recall_score)

roc_auc_score = metrics.roc_auc_score(y_test_12, pred_12)
print("roc_auc_score:   %0.3f" % roc_auc_score)
fpr, tpr, thresh = metrics.roc_curve(y_test_12, pred_12)

print("classification report:")
print(metrics.classification_report(y_test_12, pred_12))

print("confusion matrix:")
print(metrics.confusion_matrix(y_test_12, pred_12))

print('*'*15 + ' Results on Validation set ' +'*'*15, '\n')
pred_0_12 = model_lr_12.predict(X_val_12)

score = metrics.accuracy_score(y_val_12, pred_0_12)
print("accuracy:   %0.3f" % score)

f1_score = metrics.f1_score(y_val_12, pred_0_12)
print("f1_score:   %0.3f" % f1_score)

precision_score = metrics.precision_score(y_val_12, pred_0_12)
print("precision_score:   %0.3f" % precision_score)

recall_score = metrics.recall_score(y_val_12, pred_0_12)
print("recall_score:   %0.3f" % recall_score)

roc_auc_score = metrics.roc_auc_score(y_val_12, pred_0_12)
print("roc_auc_score:   %0.3f" % roc_auc_score)
fpr, tpr, thresh = metrics.roc_curve(y_val_12, pred_0_12)

print("classification report:")
print(metrics.classification_report(y_val_12, pred_0_12))

print("confusion matrix:")
print(metrics.confusion_matrix(y_val_12, pred_0_12))
print('*'*25 + ' END! ' +'*'*25)

*************** Results on Test set ***************
accuracy:   0.998
f1_score:   0.987
precision_score:   0.993
recall_score:   0.982
roc_auc_score:   0.991
classification report:
             precision    recall  f1-score   support

      False       1.00      1.00      1.00    804887
       True       0.99      0.98      0.99     81450

avg / total       1.00      1.00      1.00    886337

confusion matrix:
[[804313    574]
 [  1487  79963]]
*************** Results on Validation set *************** 

accuracy:   0.998
f1_score:   0.988
precision_score:   0.993
recall_score:   0.982
roc_auc_score:   0.991
classification report:
             precision    recall  f1-score   support

      False       1.00      1.00      1.00    804596
       True       0.99      0.98      0.99     81742

avg / total       1.00      1.00      1.00    886338

confusion matrix:
[[804068    528]
 [  1447  80295]]
************************* END! *************************


In [21]:
print('*'*15 + ' Results on the entire set ' +'*'*15, '\n')
pred__12 = model_lr_12.predict(X_12)

score = metrics.accuracy_score(y_12, pred__12)
print("accuracy:   %0.3f" % score)

f1_score = metrics.f1_score(y_12, pred__12)
print("f1_score:   %0.3f" % f1_score)

precision_score = metrics.precision_score(y_12, pred__12)
print("precision_score:   %0.3f" % precision_score)

recall_score = metrics.recall_score(y_12, pred__12)
print("recall_score:   %0.3f" % recall_score)

roc_auc_score = metrics.roc_auc_score(y_12, pred__12)
print("roc_auc_score:   %0.3f" % roc_auc_score)
fpr, tpr, thresh = metrics.roc_curve(y_12, pred__12)

print("classification report:")
print(metrics.classification_report(y_12, pred__12))

print("confusion matrix:")
print(metrics.confusion_matrix(y_12, pred__12))
print('*'*25 + ' END! ' +'*'*25)

*************** Results on the entire set *************** 

accuracy:   0.998
f1_score:   0.988
precision_score:   0.993
recall_score:   0.983
roc_auc_score:   0.991
classification report:
             precision    recall  f1-score   support

      False       1.00      1.00      1.00   8048081
       True       0.99      0.98      0.99    815294

avg / total       1.00      1.00      1.00   8863375

confusion matrix:
[[8042709    5372]
 [  13982  801312]]
************************* END! *************************


Cross validate the model on the validation set using 

1) accuracy,  <br>
  2) f1_macro,  <br>
  3) recall, and <br>
  4) precision <br>asscoring methods

In [22]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model_lr_12, X_val_12, y_val_12, scoring = 'f1_macro', cv = 10)
print(scores)
print('Mean f1 score of the 10 scores: %s'%(round(100*scores.mean(), 2)))

C:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
C:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
C:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
C:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
C:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs

[ 0.9873935   0.9885395   0.98861575  0.9868803   0.98824956  0.98830399
  0.98849316  0.98781979  0.98818651  0.98856528]
Mean f1 score of the 10 scores: 98.81


In [23]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model_lr_12, X_val_12, y_val_12, scoring = 'accuracy', cv = 10)
print(scores)
print('Mean accuracy score of the 10 scores: %s'%(round(100*scores.mean(), 2)))

C:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
C:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
C:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
C:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
C:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs

[ 0.99583686  0.99620917  0.99624298  0.99566758  0.99611887  0.99613015
  0.9961978   0.99597215  0.99609626  0.99622037]
Mean accuracy score of the 10 scores: 99.61


In [24]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model_lr_12, X_val_12, y_val_12, scoring = 'recall', cv = 10)
print(scores)
print('Mean recall score of the 10 scores: %s'%(round(100*scores.mean(), 2)))

C:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
C:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
C:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
C:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
C:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs

[ 0.96195719  0.96574924  0.96354294  0.96109616  0.96378762  0.96574505
  0.96464399  0.96427698  0.96415464  0.96513335]
Mean recall score of the 10 scores: 96.4


In [25]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model_lr_12, X_val_12, y_val_12, scoring = 'precision', cv = 10)
print(scores)
print('Mean precision score of the 10 scores: %s'%(round(100*scores.mean(), 2)))

C:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
C:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
C:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
C:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
C:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs

[ 0.99267862  0.99295686  0.99557578  0.99166877  0.99394398  0.99208244
  0.99394933  0.99182081  0.99331989  0.99370198]
Mean precision score of the 10 scores: 99.32


Add columns for the predictions and the probability of prediction for each title.

In [26]:
data['Predictions'], data['Pred_Prob0'], data['Pred_Prob1']= model_lr_12.predict(X_12), model_lr_12.predict_proba(X_12)[:,0], model_lr_12.predict_proba(X_12)[:,1] 

Save the false Positives and false negatives to a csv file for further analysis.

In [29]:
data[(data['Modified_IsSecurity'] == True) & (data['Predictions'] == False)][['Title','SourceLink','Modified_IsSecurity','Predictions','Pred_Prob0','Pred_Prob1']].to_csv('False Negatives bug,isue,compileETC06-07-18.csv', index = False)
data[(data['Modified_IsSecurity'] == False) & (data['Predictions'] == True)][['Title','SourceLink','Modified_IsSecurity','Predictions','Pred_Prob0','Pred_Prob1']].to_csv('False Positives bug,isue,compileETC06-07-18.csv', index = False)

Save the tfidf and the the model for future references

In [30]:
from sklearn.externals import joblib
joblib.dump(model_lr_12, 'model_lr_12_bugs_rdbugs_isuesETC.pkl')
joblib.dump(tv_12, 'tv_12_bugs_rdbugs_isuesETC.pkl')

['tv_12_bugs_rdbugs_isuesETC.pkl']

# Working with the whole dataset <br>(by removing the where condition in the query)
<br>We will also add "Type" to the rows being selected here because we want to see how well a new model will generalize.

In [32]:
import pyodbc
conn = pyodbc.connect(r'Driver={SQL Server};Server=uatdotdatarpt;Database=DailyMLResults;Trusted_Connection=yes;OPTION=3;applicationintent=readonly;')
data = pd.read_sql("""SELECT Title, SourceLink, Modified_IsSecurity, Type FROM MSSecurity_3 WITH (NOLOCK)
        WHERE AreaPath NOT LIKE '%RD\OneComplianceTest%' 
        AND AreaPath NOT LIKE '%\ClientManagement%'
        AND (IsDataDraft = 0 OR IsDataDraft IS NULL) 

UNION
SELECT Title, SourceLink, Modified_IsSecurity, Type FROM Not_MSSecurity_3 WITH (NOLOCK)
        WHERE AreaPath NOT LIKE '%RD\OneComplianceTest%' 
        AND AreaPath NOT LIKE '%\ClientManagement%'
        AND (IsDataDraft = 0 OR IsDataDraft IS NULL) 

UNION
SELECT Title, SourceLink, Modified_IsSecurity, Type FROM Not_Labeled_3 WITH (NOLOCK)
        WHERE AreaPath NOT LIKE '%RD\OneComplianceTest%' 
        AND AreaPath NOT LIKE '%\ClientManagement%'
        AND (IsDataDraft = 0 OR IsDataDraft IS NULL ) """, conn)
conn.close()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15890011 entries, 0 to 15890010
Data columns (total 4 columns):
Title                  object
SourceLink             object
Modified_IsSecurity    bool
Type                   object
dtypes: bool(1), object(3)
memory usage: 378.8+ MB


In [33]:
data['Title'] = data['Title'].apply(str)

In [34]:
data['Title'] = data['Title'].apply(deal_with_policheck)

In [35]:
data['No Urls'] = data['Title'].apply(clean_review_html)

In [36]:
data['No Paths'] = data['No Urls'].apply(clean_review_path)

In [37]:
data['No Links and Paths'] = data['No Paths'].apply(remove_paths_urls)

In [38]:
data['Split Upper'] = data['No Links and Paths'].apply(split_on_uppercase)

In [39]:
data['No Punctuation'] = data['Split Upper'].apply(remove_punctuation)

In [40]:
data['Clean Title'] = data['No Punctuation'].apply(remove_integers)

Get a tfidf and fit a logistic regression model.

In [41]:
X_12, y_12 = data['Clean Title'], data['Modified_IsSecurity']
tv_12 = TfidfVectorizer(min_df = 0.00009, max_df = 0.5, use_idf = True, stop_words = 'english', ngram_range = (1,2))
X_12 = tv_12.fit_transform(X_12)
X_train_12, X_12_0, y_train_12, y_12_0 = train_test_split(X_12, y_12, test_size = 0.2, random_state = 42)
X_test_12, X_val_12, y_test_12, y_val_12 = train_test_split(X_12_0, y_12_0, test_size = 0.5, random_state = 42)
vocab_12 = tv_12.get_feature_names()
print(len(vocab_12))

13835


In [42]:
model_lr_12 = LogisticRegression(n_jobs = -1)
model_lr_12.fit(X_train_12, y_train_12)

C:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

Get predictions on the training and validation sets

In [43]:
print('*'*15 + ' Results on Test set ' +'*'*15)
pred_12 = model_lr_12.predict(X_test_12)

score = metrics.accuracy_score(y_test_12, pred_12)
print("accuracy:   %0.3f" % score)

f1_score = metrics.f1_score(y_test_12, pred_12)
print("f1_score:   %0.3f" % f1_score)

precision_score = metrics.precision_score(y_test_12, pred_12)
print("precision_score:   %0.3f" % precision_score)

recall_score = metrics.recall_score(y_test_12, pred_12)
print("recall_score:   %0.3f" % recall_score)

roc_auc_score = metrics.roc_auc_score(y_test_12, pred_12)
print("roc_auc_score:   %0.3f" % roc_auc_score)
fpr, tpr, thresh = metrics.roc_curve(y_test_12, pred_12)

print("classification report:")
print(metrics.classification_report(y_test_12, pred_12))

print("confusion matrix:")
print(metrics.confusion_matrix(y_test_12, pred_12))

print('*'*15 + ' Results on Validation set ' +'*'*15, '\n')
pred_0_12 = model_lr_12.predict(X_val_12)

score = metrics.accuracy_score(y_val_12, pred_0_12)
print("accuracy:   %0.3f" % score)

f1_score = metrics.f1_score(y_val_12, pred_0_12)
print("f1_score:   %0.3f" % f1_score)

precision_score = metrics.precision_score(y_val_12, pred_0_12)
print("precision_score:   %0.3f" % precision_score)

recall_score = metrics.recall_score(y_val_12, pred_0_12)
print("recall_score:   %0.3f" % recall_score)

roc_auc_score = metrics.roc_auc_score(y_val_12, pred_0_12)
print("roc_auc_score:   %0.3f" % roc_auc_score)
fpr, tpr, thresh = metrics.roc_curve(y_val_12, pred_0_12)

print("classification report:")
print(metrics.classification_report(y_val_12, pred_0_12))

print("confusion matrix:")
print(metrics.confusion_matrix(y_val_12, pred_0_12))
print('*'*25 + ' END! ' +'*'*25)

*************** Results on Test set ***************
accuracy:   0.995
f1_score:   0.968
precision_score:   0.988
recall_score:   0.949
roc_auc_score:   0.974
classification report:
             precision    recall  f1-score   support

      False       1.00      1.00      1.00   1464204
       True       0.99      0.95      0.97    124797

avg / total       1.00      1.00      1.00   1589001

confusion matrix:
[[1462749    1455]
 [   6394  118403]]
*************** Results on Validation set *************** 

accuracy:   0.995
f1_score:   0.968
precision_score:   0.988
recall_score:   0.949
roc_auc_score:   0.974
classification report:
             precision    recall  f1-score   support

      False       1.00      1.00      1.00   1464600
       True       0.99      0.95      0.97    124402

avg / total       1.00      1.00      1.00   1589002

confusion matrix:
[[1463190    1410]
 [   6399  118003]]
************************* END! *************************


Get predictions on the entire dataset

In [44]:
print('*'*15 + ' Results on the entire set ' +'*'*15, '\n')
pred__12 = model_lr_12.predict(X_12)

score = metrics.accuracy_score(y_12, pred__12)
print("accuracy:   %0.3f" % score)

f1_score = metrics.f1_score(y_12, pred__12)
print("f1_score:   %0.3f" % f1_score)

precision_score = metrics.precision_score(y_12, pred__12)
print("precision_score:   %0.3f" % precision_score)

recall_score = metrics.recall_score(y_12, pred__12)
print("recall_score:   %0.3f" % recall_score)

roc_auc_score = metrics.roc_auc_score(y_12, pred__12)
print("roc_auc_score:   %0.3f" % roc_auc_score)
fpr, tpr, thresh = metrics.roc_curve(y_12, pred__12)

print("classification report:")
print(metrics.classification_report(y_12, pred__12))

print("confusion matrix:")
print(metrics.confusion_matrix(y_12, pred__12))
print('*'*25 + ' END! ' +'*'*25)

*************** Results on the entire set *************** 

accuracy:   0.995
f1_score:   0.968
precision_score:   0.988
recall_score:   0.950
roc_auc_score:   0.974
classification report:
             precision    recall  f1-score   support

      False       1.00      1.00      1.00  14641962
       True       0.99      0.95      0.97   1248049

avg / total       1.00      1.00      1.00  15890011

confusion matrix:
[[14627800    14162]
 [   62991  1185058]]
************************* END! *************************


In [55]:
62991/1245058

0.050592823788128745

Perform a 10-fold cross-balidation and print the results.

In [45]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model_lr_12, X_val_12, y_val_12, scoring = 'recall', cv = 10)
print(scores)
print('Mean recall score of the 10 scores: %s'%(round(100*scores.mean(), 2)))

C:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
C:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
C:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
C:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
C:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs

[ 0.93336549  0.93457118  0.9335209   0.93247588  0.93802251  0.93625402
  0.93649518  0.93842444  0.93536977  0.93456592]
Mean recall score of the 10 scores: 93.53


In [46]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model_lr_12, X_val_12, y_val_12, scoring = 'accuracy', cv = 10)
print(scores)
print('Mean accuracy score of the 10 scores: %s'%(round(100*scores.mean(), 2)))

C:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
C:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
C:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
C:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
C:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs

[ 0.99399626  0.9939648   0.99394588  0.99381372  0.99431718  0.99405286
  0.99415356  0.99438011  0.99409692  0.99408433]
Mean accuracy score of the 10 scores: 99.41


In [47]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model_lr_12, X_val_12, y_val_12, scoring = 'precision', cv = 10)
print(scores)
print('Mean precision score of the 10 scores: %s'%(round(100*scores.mean(), 2)))

C:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
C:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
C:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
C:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
C:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs

[ 0.98934992  0.98768264  0.98850868  0.98782253  0.98881451  0.98711755
  0.98820935  0.9892382   0.98861512  0.98927842]
Mean precision score of the 10 scores: 98.85


In [48]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model_lr_12, X_val_12, y_val_12, scoring = 'f1_macro', cv = 10)
print(scores)
print('Mean f1_macro score of the 10 scores: %s'%(round(100*scores.mean(), 2)))

C:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
C:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
C:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
C:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))
C:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs

[ 0.97864679  0.97856335  0.97847589  0.97800186  0.97983651  0.97889708
  0.97924652  0.9800598   0.97903037  0.97897132]
Mean f1_macro score of the 10 scores: 97.9


In [49]:
data.columns

Index(['Title', 'SourceLink', 'Modified_IsSecurity', 'Type', 'No Urls',
       'No Paths', 'No Links and Paths', 'Split Upper', 'No Punctuation',
       'Clean Title'],
      dtype='object')

Get the predictions for each title and the probability attributed to that prediction.

In [50]:
data['Predictions'], data['Pred_Proba0'], data['Pred_Proba1']=model_lr_12.predict(X_12), model_lr_12.predict_proba(X_12)[:,0], model_lr_12.predict_proba(X_12)[:,1] 

In [52]:
fn,fp = data[(data['Modified_IsSecurity'] == True) & (data['Predictions'] == False)][['Title', 'SourceLink','Modified_IsSecurity','Predictions','Pred_Proba0','Pred_Proba1','Type']], data[(data['Modified_IsSecurity'] == False) & (data['Predictions'] == True)][['Title', 'SourceLink','Modified_IsSecurity','Predictions','Pred_Proba0','Pred_Proba1','Type']]

Group all the Titles, false negatives and false positives by type

In [73]:
grouped_type = pd.DataFrame(data.groupby(by = ['Type']).count()['Title'])
grouped_type_fp = pd.DataFrame(fp.groupby(by = ['Type']).count()['Title'])
grouped_type_fn = pd.DataFrame(fn.groupby(by = ['Type']).count()['Title'])

In [71]:
grouped_type.columns

Index(['Title'], dtype='object')

In [110]:
grouped_type_fn.head()

,Title
Type,
Assessment,11
Bug,13307
Criterion,23
Customer Promise,41
Defect,2941


In [75]:
grouped_type.index

Index(['Accessibility Exemption', 'Accessibility Plan', 'Assessment', 'BUG',
       'Blanket Exception', 'Bug', 'Compliance Review', 'Criterion',
       'Customer Ask', 'Customer Promise', 'DTS Task', 'Defect', 'Deliverable',
       'Dependency', 'Design Requirement', 'Design Task', 'Epic', 'Exception',
       'Experience', 'External Dependency', 'Feature', 'Feature Release',
       'FeedbackTicket', 'Icon Request', 'Impediment', 'Improvement', 'Issue',
       'Live Site Incident', 'Partner Ask', 'Product Backlog Item', 'RDBug',
       'RDTask', 'ReadMe Input', 'Release Exception', 'Release Record',
       'Release Security and Fundamentals Checklist', 'Requirement', 'Roadmap',
       'Scenario', 'Security Checklist', 'Service Catalog', 'Service Change',
       'Slice', 'Story', 'TASK', 'Task', 'Task Group', 'UI Component',
       'User Story', 'VSEng Service Incident', 'Work Item'],
      dtype='object', name='Type')

Concatenate all three dataframes

In [80]:
concat_grouped_by = pd.concat([grouped_type, grouped_type_fn, grouped_type_fp], axis = 1)
concat_grouped_by.info()

<class 'pandas.core.frame.DataFrame'>
Index: 51 entries, Accessibility Exemption to Work Item
Data columns (total 3 columns):
Title    51 non-null int64
Title    33 non-null float64
Title    27 non-null float64
dtypes: float64(2), int64(1)
memory usage: 1.6+ KB


In [82]:
concat_grouped_by.columns=['Title', 'fn_title', 'fp_title']
concat_grouped_by.head()

,Title,fn_title,fp_title
Accessibility Exemption,15,NaN,NaN
Accessibility Plan,116,NaN,2.0
Assessment,8223,11.0,21.0
BUG,274,NaN,NaN
Blanket Exception,28,NaN,NaN


In [87]:
concat_grouped_by['Total_Type'] = concat_grouped_by.sum()
concat_grouped_by.head()

,Title,fn_title,fp_title,Total_Type
Accessibility Exemption,15,NaN,NaN,NaN
Accessibility Plan,116,NaN,2.0,NaN
Assessment,8223,11.0,21.0,NaN
BUG,274,NaN,NaN,NaN
Blanket Exception,28,NaN,NaN,NaN


Create a column for the percentage of false negatives and false positives for each work Item type

In [102]:
concat_grouped_by['Percent_fn'] = round(100*concat_grouped_by['fn_title']/concat_grouped_by['Title'],3)
concat_grouped_by['Percent_fp'] = round(100*concat_grouped_by['fp_title']/concat_grouped_by['Title'],3)
concat_grouped_by.head()

,Title,fn_title,fp_title,Total_Type,Percent_fn,Percent_fp
Accessibility Exemption,15,NaN,NaN,NaN,NaN,NaN
Accessibility Plan,116,NaN,2.0,NaN,NaN,1.724
Assessment,8223,11.0,21.0,NaN,0.134,0.255
BUG,274,NaN,NaN,NaN,NaN,NaN
Blanket Exception,28,NaN,NaN,NaN,NaN,NaN


Rename the columns to more intuitive names and checkout the head of the resulting dataframe.

In [105]:
concat_grouped_by.rename(inplace = True, columns={'Title':'numTitles', 'fn_title':'numFN', 'fp_title':'numFP', 'Total_Type':'Total_Type', 'Percent_fn':'Percent_fn','Percent_fp':'Percent_fp'})

In [107]:
concat_grouped_by.drop('Total_Type', inplace = True, axis = 1)

In [113]:
concat_grouped_by.head()

,numTitles,numFN,numFP,Percent_fn,Percent_fp
Accessibility Exemption,15,NaN,NaN,NaN,NaN
Accessibility Plan,116,NaN,2.0,NaN,1.724
Assessment,8223,11.0,21.0,0.134,0.255
BUG,274,NaN,NaN,NaN,NaN
Blanket Exception,28,NaN,NaN,NaN,NaN


Save the grouped data to csv files for further analylis

In [109]:
concat_grouped_by.to_csv('False Negatives and false Positives grouped by workItem Type.csv')

Save the false negatives and false positives of this model to a csv file for further anlysis.

In [114]:
fn.to_csv('False Negatives all work item type 06-08-2018.csv', index = False)
fp.to_csv('False Positives all work item type 06-08-2018.csv', index = False)

In [115]:
fp.head()

,Title,SourceLink,Modified_IsSecurity,Predictions,Pred_Proba0,Pred_Proba1,Type
2403,(IQP) A certain SecureIslands API call not wo...,https://msazure.visualstudio.com/DefaultCollec...,False,True,0.099604,0.900396,Bug
4797,[PPE] 404's for GetServerSyncUpdateDecryption...,https://microsoft.visualstudio.com/DefaultColl...,False,True,0.039426,0.960574,Bug
6963,------------------- Infrastructure Cutline = O...,https://o365exchange.visualstudio.com/DefaultC...,False,True,0.039439,0.960561,User Story
7462,🛂 Configure RBAC,https://msengagement.visualstudio.com/DefaultC...,False,True,0.276454,0.723546,Feature
8882,Develop standard access control testing crite...,https://microsoftit.visualstudio.com/DefaultCo...,False,True,0.364225,0.635775,Task


Save the logistic regression model and its tfidf

In [116]:
joblib.dump(model_lr_12, 'model_lr_12_allWorkItemTypes.pkl')
joblib.dump(tv_12, 'tv_12_allWorkItemTypes.pkl')

['tv_12_allWorkItemTypes.pkl']

# Working on the "msasg" data
Get predictions for the msasg data

Get the data

In [124]:
conn = pyodbc.connect(r'Driver={SQL Server};Server=dotdatarpt;Database=TwCDataWarehouse;Trusted_Connection=yes;OPTION=3;applicationintent=readonly;')

msasg_data = pd.read_sql('''SELECT DISTINCT Title, SourceLink, RequirementCollectionDID 
                          FROM workitemextensions WITH (NOLOCK)
                          WHERE IsActive=1 and LatestFlag=1 
                          AND SourceLink like '%msasg.visualstudio.com%' ''', conn)
conn.close()
# and Type LIKE '%bug%' 

In [125]:
msasg_data.head()

,Title,SourceLink,RequirementCollectionDID
0,None,https://msasg.visualstudio.com/DefaultCollecti...,None
1,None,https://msasg.visualstudio.com/DefaultCollecti...,None
2,None,https://msasg.visualstudio.com/DefaultCollecti...,None
3,None,https://msasg.visualstudio.com/DefaultCollecti...,None
4,None,https://msasg.visualstudio.com/DefaultCollecti...,None


In [126]:
msasg_data.tail()

,Title,SourceLink,RequirementCollectionDID
1071257,首页和verp页脚加QQ群号,https://msasg.visualstudio.com/DefaultCollecti...,None
1071258,駄菓子クーポン設置,https://msasg.visualstudio.com/DefaultCollecti...,None
1071259,高德离线地图实现方式调研报告,https://msasg.visualstudio.com/DefaultCollecti...,None
1071260,高校简介 data,https://msasg.visualstudio.com/DefaultCollecti...,None
1071261,\tLabel Guideline update,https://msasg.visualstudio.com/DefaultCollecti...,None


Preprocess the data

In [127]:
msasg_data['Title'] = msasg_data['Title'].apply(str)


In [128]:
msasg_data['Title'] = msasg_data['Title'].apply(deal_with_policheck)


In [129]:
msasg_data['No Urls'] = msasg_data['Title'].apply(clean_review_html)


In [130]:
msasg_data['No Paths'] = msasg_data['No Urls'].apply(clean_review_path)


In [131]:
msasg_data['No Links and Paths'] = msasg_data['No Paths'].apply(remove_paths_urls)


In [132]:
msasg_data['Split Upper'] = msasg_data['No Links and Paths'].apply(split_on_uppercase)


In [133]:
msasg_data['No Punctuation'] = msasg_data['Split Upper'].apply(remove_punctuation)


In [134]:
msasg_data['Clean Title'] = msasg_data['No Punctuation'].apply(remove_integers)

In [135]:
X = tv_12.transform(msasg_data['No Punctuation'])

In [136]:
msasg_data['Predictions'], msasg_data['Pred_Proba0'], msasg_data['Pred_Proba1'] = model_lr_12.predict(X), model_lr_12.predict_proba(X)[:,0], model_lr_12.predict_proba(X)[:,1] 

In [137]:
msasg_data['Predictions'].value_counts()

False    1033208
True       38054
Name: Predictions, dtype: int64

In [162]:
msasg_data['Predictions_fewTypes'].value_counts()

False    1033709
True       37553
Name: Predictions_fewTypes, dtype: int64

In [145]:
print('Percentage predicted as security: %f'%(round(100* 38054/(38054+1033208), 5))+'%.')

Percentage predicted as security: 3.552260%.


Get predictions for the msasg titles using model built on <br>
<br> `Type IN ('bug', 'Compliance Issue', 'Defect', 'Issue', 'RDBug')`

In [147]:
model = joblib.load('model_lr_12_bugs_rdbugs_isuesETC.pkl')
tv = joblib.load('tv_12_bugs_rdbugs_isuesETC.pkl')

In [148]:
X1 = tv.transform(msasg_data['No Punctuation'])

In [150]:
msasg_data['Predictions_fewTypes'], msasg_data['Pred_Proba0_fewTypes'], msasg_data['Pred_Proba1_fewTypes'] = model.predict(X1), model.predict_proba(X1)[:,0], model.predict_proba(X1)[:,1] 

Get the predictions where the model built on all work items types do not agree with the model built on work <br>items with `Type IN ('bug', 'Compliance Issue', 'Defect', 'Issue', 'RDBug')`

In [151]:
msasg_data[msasg_data['Predictions'] != msasg_data['Predictions_fewTypes']]

,Title,SourceLink,RequirementCollectionDID,No Urls,No Paths,No Links and Paths,Split Upper,No Punctuation,Clean Title,Predictions,Pred_Proba0,Pred_Proba1,Predictions_fewTypes,Pred_Proba0_fewTypes,Pred_Proba1_fewTypes
6575,(Via External Team) Verify Decryption is Worki...,https://msasg.visualstudio.com/DefaultCollecti...,MS.Security,(Via External Team) Verify Decryption is Worki...,(Via External Team) Verify Decryption is Worki...,(Via External Team) Verify Decryption is Worki...,( Via External Team) Verify Decryption is ...,Via External Team Verify Decryption is Working...,Via External Team Verify Decryption is Working...,True,0.042940,0.957060,False,0.997046,0.002954
6914,*RELEASE BLOCKER* - Secret not working between...,https://msasg.visualstudio.com/DefaultCollecti...,None,*RELEASE BLOCKER* - Secret not working between...,*RELEASE BLOCKER* - Secret not working between...,*RELEASE BLOCKER* - Secret not working between...,*RELEASE BLOCKER* - Secret not working betwee...,RELEASE BLOCKER Secret not working between Fal...,RELEASE BLOCKER Secret not working between Fal...,True,0.395128,0.604872,False,0.635401,0.364599
6917,*RELEASE BLOCKER* Add secret token communicati...,https://msasg.visualstudio.com/DefaultCollecti...,None,*RELEASE BLOCKER* Add secret token communicati...,*RELEASE BLOCKER* Add secret token communicati...,*RELEASE BLOCKER* Add secret token communicati...,*RELEASE BLOCKER* Add secret token communicat...,RELEASE BLOCKER Add secret token communication...,RELEASE BLOCKER Add secret token communication...,True,0.345625,0.654375,False,0.561229,0.438771
6936,*RELEASE BLOCKER* Setup secret between SNR/XAP...,https://msasg.visualstudio.com/DefaultCollecti...,None,*RELEASE BLOCKER* Setup secret between SNR/XAP...,*RELEASE BLOCKER* Setup secret between SNR/XAP...,*RELEASE BLOCKER* Setup secret between and rem...,*RELEASE BLOCKER* Setup secret between and re...,RELEASE BLOCKER Setup secret between and remov...,RELEASE BLOCKER Setup secret between and remov...,True,0.490574,0.509426,False,0.683140,0.316860
15224,[ MSASG_AdsCX_retail ][ CodeAnalysis ] - Defec...,https://msasg.visualstudio.com/DefaultCollecti...,None,[ MSASG_AdsCX_retail ][ CodeAnalysis ] - Defec...,[ MSASG_AdsCX_retail ][ CodeAnalysis ] - Defec...,[ MSASG_AdsCX_retail ][ CodeAnalysis ] - Defec...,[ MSASG_ Ads CX_retail ][ Code Analysis ] - ...,MSASG Ads CX retail Code Analysis Defect Revie...,MSASG Ads CX retail Code Analysis Defect Revie...,False,0.681045,0.318955,True,0.448253,0.551747
15225,[ MSASG_AdsCX_retail ][ CodeAnalysis ] - Defec...,https://msasg.visualstudio.com/DefaultCollecti...,None,[ MSASG_AdsCX_retail ][ CodeAnalysis ] - Defec...,[ MSASG_AdsCX_retail ][ CodeAnalysis ] - Defec...,[ MSASG_AdsCX_retail ][ CodeAnalysis ] - Defec...,[ MSASG_ Ads CX_retail ][ Code Analysis ] - ...,MSASG Ads CX retail Code Analysis Defect Revie...,MSASG Ads CX retail Code Analysis Defect Revie...,False,0.681045,0.318955,True,0.448253,0.551747
15937,[ MSASG_Agilemining-Packages_retail ][ CodeAna...,https://msasg.visualstudio.com/DefaultCollecti...,None,[ MSASG_Agilemining-Packages_retail ][ CodeAna...,[ MSASG_Agilemining-Packages_retail ][ CodeAna...,[ MSASG_Agilemining-Packages_retail ][ CodeAna...,[ MSASG_ Agilemining- Packages_retail ][ Code...,MSASG Agilemining Packages retail Code Analysi...,MSASG Agilemining Packages retail Code Analysi...,False,0.606666,0.393334,True,0.458998,0.541002
15972,[ MSASG_AnB_retail ][ CodeAnalysis ] - Defect ...,https://msasg.visualstudio.com/DefaultCollecti...,None,[ MSASG_AnB_retail ][ CodeAnalysis ] - Defect ...,[ MSASG_AnB_retail ][ CodeAnalysis ] - Defect ...,[ MSASG_AnB_retail ][ CodeAnalysis ] - Defect ...,[ MSASG_ An B_retail ][ Code Analysis ] - De...,MSASG An B retail Code Analysis Defect Transpa...,MSASG An B retail Code Analysis Defect Transpa...,False,0.515939,0.484061,True,0.261921,0.738079
21996,[ MSASG_Indexquality_Malware_Prod_retail ][ Co...,https://msasg.visualstudio.com/DefaultCollecti...,None,[ MSASG_Indexqualit

In [152]:
msasg_data.columns

Index(['Title', 'SourceLink', 'RequirementCollectionDID', 'No Urls',
       'No Paths', 'No Links and Paths', 'Split Upper', 'No Punctuation',
       'Clean Title', 'Predictions', 'Pred_Proba0', 'Pred_Proba1',
       'Predictions_fewTypes', 'Pred_Proba0_fewTypes', 'Pred_Proba1_fewTypes'],
      dtype='object')

In [153]:
msasg_data[msasg_data['Predictions'] != msasg_data['Predictions_fewTypes']][['Title', 'SourceLink', 
                                                                             'RequirementCollectionDID', 
                                                                             'Predictions', 
                                                                             'Pred_Proba0', 'Pred_Proba1',
                                                                             'Predictions_fewTypes', 
                                                                             'Pred_Proba0_fewTypes', 
                                                                             'Pred_Proba1_fewTypes']].to_csv('msasg titles where both models disagree on prediction.csv')

In [157]:
msasg_data[msasg_data['Predictions'] != msasg_data['Predictions_fewTypes']][['Clean Title', 'Predictions','Predictions_fewTypes']].sort() 

,Clean Title,Predictions,Predictions_fewTypes
6575,Via External Team Verify Decryption is Working...,True,False
6914,RELEASE BLOCKER Secret not working between Fal...,True,False
6917,RELEASE BLOCKER Add secret token communication...,True,False
6936,RELEASE BLOCKER Setup secret between and remov...,True,False
15224,MSASG Ads CX retail Code Analysis Defect Revie...,False,True
15225,MSASG Ads CX retail Code Analysis Defect Revie...,False,True
15937,MSASG Agilemining Packages retail Code Analysi...,False,True
15972,MSASG An B retail Code Analysis Defect Transpa...,False,True
21996,MSASG Indexquality Malware Prod retail Code An...,True,False
33656,Scheduling Setting up Secure Service Fabric Cl...,True,False


In [155]:
[True for word in 'Aether Secure Implement GC for Services'.lower().split() if word in vocab_12]

[True, True, True, True, True]

In [161]:
msasg_data.sort_values(['Clean Title'],ascending = True)

,Title,SourceLink,RequirementCollectionDID,No Urls,No Paths,No Links and Paths,Split Upper,No Punctuation,Clean Title,Predictions,Pred_Proba0,Pred_Proba1,Predictions_fewTypes,Pred_Proba0_fewTypes,Pred_Proba1_fewTypes
7081,//Build,https://msasg.visualstudio.com/DefaultCollecti...,None,//Build,//Build,,,,,False,0.996670,0.003330,False,0.998447,0.001553
278811,{},https://msasg.visualstudio.com/DefaultCollecti...,None,{},{},{},{},,,False,0.996670,0.003330,False,0.998447,0.001553
278810,{},https://msasg.visualstudio.com/DefaultCollecti...,None,{},{},{},{},,,False,0.996670,0.003330,False,0.998447,0.001553
278809,{},https://msasg.visualstudio.com/DefaultCollecti...,None,{},{},{},{},,,False,0.996670,0.003330,False,0.998447,0.001553
278808,{},https://msasg.visualstudio.com/DefaultCollecti...,None,{},{},{},{},,,False,0.996670,0.003330,False,0.998447,0.001553
278807,{},https://msasg.visualstudio.com/DefaultCollecti...,None,{},{},{},{},,,False,0.996670,0.003330,False,0.998447,0.001553
278806,{},https://msasg.visualstudio.com/DefaultCollecti...,None,{},{},{},{},,,False,0.996670,0.003330,False,0.998447,0.001553
278805,{},https://msasg.visualstudio.com/DefaultCollecti...,None,{},{},{},{},,,False,0.996670,0.003330,False,0.998447,0.001553
278804,{},https://msasg.visualstudio.com/DefaultCollecti...,None,{},{},{},{},,,False,0.996670,0.003330,False,0.998447,0.001553
278803,{},https://msasg.visualstudio.com/DefaultCollecti...,None,{},{},{},{},,,False,0.996670,0.003330,False,0.998447,0.001553


Get the msasf titles with the same type condition as that in the query that we had used in building the first moded ie <br>  `Type IN ('bug', 'Compliance Issue', 'Defect', 'Issue', 'RDBug')`

In [165]:
conn = pyodbc.connect(r'Driver={SQL Server};Server=dotdatarpt;Database=TwCDataWarehouse;Trusted_Connection=yes;OPTION=3;applicationintent=readonly;')

msasg_data1 = pd.read_sql('''SELECT DISTINCT Title, SourceLink, RequirementCollectionDID 
                          FROM workitemextensions WITH (NOLOCK)
                          WHERE IsActive = 1 AND LatestFlag = 1 
                          AND Type IN ('bug', 'Compliance Issue', 'Defect', 'Issue', 'RDBug')
                          AND SourceLink like '%msasg.visualstudio.com%' ''', conn)
conn.close()
msasg_data1.info()
# and Type LIKE '%bug%' 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 457700 entries, 0 to 457699
Data columns (total 3 columns):
Title                       457700 non-null object
SourceLink                  457700 non-null object
RequirementCollectionDID    29295 non-null object
dtypes: object(3)
memory usage: 10.5+ MB


In [166]:
msasg_data1['Title'] = msasg_data1['Title'].apply(str)


In [167]:
msasg_data1['Title'] = msasg_data1['Title'].apply(deal_with_policheck)


In [168]:
msasg_data1['No Urls'] = msasg_data1['Title'].apply(clean_review_html)


In [169]:
msasg_data1['No Paths'] = msasg_data1['No Urls'].apply(clean_review_path)


In [170]:
msasg_data1['No Links and Paths'] = msasg_data1['No Paths'].apply(remove_paths_urls)


In [171]:
msasg_data1['Split Upper'] = msasg_data1['No Links and Paths'].apply(split_on_uppercase)


In [172]:
msasg_data1['No Punctuation'] = msasg_data1['Split Upper'].apply(remove_punctuation)


In [173]:
msasg_data1['Clean Title'] = msasg_data1['No Punctuation'].apply(remove_integers)

Transform the 'No Punctuation' column and get predictions on it.

In [175]:
X2 = tv.transform(msasg_data1['No Punctuation'])

Get the predictions on the transformed titles.

In [176]:
msasg_data1['Predictions_fewTypes'], msasg_data1['Pred_Proba0_fewTypes'], msasg_data1['Pred_Proba1_fewTypes'] = model.predict(X2), model.predict_proba(X2)[:,0], model.predict_proba(X2)[:,1] 

In [177]:
msasg_data.columns

Index(['Title', 'SourceLink', 'RequirementCollectionDID', 'No Urls',
       'No Paths', 'No Links and Paths', 'Split Upper', 'No Punctuation',
       'Clean Title', 'Predictions_fewTypes', 'Pred_Proba0_fewTypes',
       'Pred_Proba1_fewTypes'],
      dtype='object')

In [ ]:
['Title', 'SourceLink', 'RequirementCollectionDID', 'No Urls',
       'No Paths', 'No Links and Paths', 'Split Upper', 'No Punctuation',
       'Clean Title', 'Predictions_fewTypes', 'Pred_Proba0_fewTypes',
       'Pred_Proba1_fewTypes']

In [178]:
msasg_data['Predictions_fewTypes'].value_counts()

False    428803
True      28897
Name: Predictions_fewTypes, dtype: int64

In [179]:
msasg_data[['Title', 'SourceLink', 'RequirementCollectionDID','Predictions_fewTypes', 'Pred_Proba0_fewTypes', 'Pred_Proba1_fewTypes']].to_csv('Predictions on msasg titles.csv.csv', index = False)